In [ ]:
import pandas as pd
import glacierml as gl
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import gaussian_kde

pd.set_option('display.max_columns',None)
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

In [ ]:
df = pd.read_pickle(os.path.join(
    coregistration_testing_path,
    '1','rgi_est_raw.pkl'
))
for i in range(2,5,1):
    dft = pd.read_pickle(
        os.path.join(
            coregistration_testing_path,str(i),'rgi_est_raw.pkl'
        )
    )
    df = pd.merge(df,dft, how = 'inner', on = list(dft)[:-1])

ref = pd.read_pickle(os.path.join(home_path,'data/reference_thicknesses/refs.pkl'))

df = pd.merge(df, ref, how = 'inner', on = 'RGIId')
df

In [ ]:
glac1 = gl.coregister_data(data_path,'1')
glac2 = gl.coregister_data(data_path,'2')
glac3 = gl.coregister_data(data_path,'3')
glac4 = gl.coregister_data(data_path,'4')

df1 = pd.merge(df,glac1, how = 'inner', on = 'RGIId')
df2 = pd.merge(df,glac2, how = 'inner', on = 'RGIId')
df3 = pd.merge(df,glac3, how = 'inner', on = 'RGIId')
df4 = pd.merge(df,glac4, how = 'inner', on = 'RGIId')

df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)

df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)

In [ ]:
fig, ax = plt.subplots(2,2,figsize = (6,6),
#                        sharex = True, sharey = True
                      )

for n, dataset in enumerate((df1, df2,
                             df3, df4
                            )):
    data = dataset
    data = data.drop(data[data['RGIId'].duplicated(keep = False)].index)

    data_1 = np.round(data[n+1],0)
    data_3 = np.round(data['Thickness'],0)

    res_mean = np.round(np.mean(data_1 - data_3),0)
    res_std = np.round(np.std(data_1-data_3),0)
    print(f'mean residual: {res_mean}\nresidual std {res_std}')
#     data_1 = data['FMT'] / 1e3 * data['Area_x']

    print('Sorting data...')
    x_1 = np.sort(data_1)
    y_1 = 1. * np.arange(len(data_1)) / (len(data_1) - 1)

    x_3 = np.sort(data_3)
    y_3 = 1. * np.arange(len(data_3)) / (len(data_3) - 1)

    print('Calculating point density...')
    print('')
    # Calculate the point density
    xy = np.vstack([np.log10(data_1),np.log10(data_3)])

    z = gaussian_kde(xy)(xy)   
    if n == 0:
        ax[0,0].scatter(
                data_3,
                data_1,
                c = z,
                cmap = 'viridis',
                marker = '.'
            )    
        ax[0,0].set_xscale('log')
        ax[0,0].set_yscale('log')
        ax[0,0].set_title(
            f'N = {len(data)}\nResidual = {res_mean:.0f}$\pm${res_std:.0f} (m)'
                         )
        ax[0,0].plot(
            (data_3.min(), data_3.max()),
            (data_3.min(), data_3.max()),
            color = 'red',
            linestyle = '-'
        )
    if n == 1:
        ax[0,1].scatter(
                data_3,
                data_1,
                c = z,
                cmap = 'viridis',
                marker = '.'
            ) 
        
        ax[0,1].set_xscale('log')
        ax[0,1].set_yscale('log')
        ax[0,1].set_title(
            f'N = {len(data)}\nResidual = {res_mean:.0f}$\pm${res_std:.0f} (m)'
                         )
        ax[0,1].plot(
            (data_3.min(), data_3.max()),
            (data_3.min(), data_3.max()),
            color = 'red',
            linestyle = '-'
        )
    if n == 2:
        ax[1,0].scatter(
                data_3,
                data_1,
                c = z,
                cmap = 'viridis',
                marker = '.'
            )    
        
        ax[1,0].set_xscale('log')
        ax[1,0].set_yscale('log')
        ax[1,0].set_title(
            f'N = {len(data)}\nResidual = {res_mean:.0f}$\pm${res_std:.0f} (m)'
        )
        ax[1,0].plot(
            (data_3.min(), data_3.max()),
            (data_3.min(), data_3.max()),
            color = 'red',
            linestyle = '-'
        )
    if n == 3:
        ax[1,1].scatter(
                data_3,
                data_1,
                c = z,
                cmap = 'viridis',
                marker = '.'
        )    
        ax[1,1].set_xscale('log')
        ax[1,1].set_yscale('log')
        ax[1,1].set_title(
            f'N = {len(data)}\nResidual = {res_mean:.0f}$\pm${res_std:.0f} (m)'
        )
        ax[1,1].plot(
            (data_3.min(), data_3.max()),
            (data_3.min(), data_3.max()),
            color = 'red',
            linestyle = '-'
        )

fig.supylabel('This Study Mean Estimated Thickness (m)')
fig.supxlabel('GlaThiDa Mean Thickness (m)')

plt.tight_layout()